# 192.039 Deep Learning for Natural Language Processing W2024
## Seq2Seq Translation German to English
Diego Anas Rejas Haddioui

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [ ]:
from typing import Tuple


def readLangs(input_file, reverse=False)-> Tuple[Lang,Lang,Tuple[str,str]]:
    print("Reading lines...")

    # Read the file and split into lines
    with open(input_file, encoding='utf-8') as f:
        lines = f.read().strip().split('\n')

    # Only keep the first two parts (English and German sentences)
    pairs = []
    for line in lines:
        parts = line.split('\t')
        if len(parts) >= 2:  # Ensure there are at least two parts
            english, german = parts[0], parts[1]
            pairs.append([normalizeString(english), normalizeString(german)])

    # Reverse pairs if necessary
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang("eng")  # German-to-English, so input is English when reversed
        output_lang = Lang("deu")
    else:
        input_lang = Lang("deu")
        output_lang = Lang("eng")

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
# [Prepare Data with Train-Test Split]
def prepareData(test_ratio=0.2, reverse=False):
    input_file = "/content/drive/MyDrive/Datasets/deu.txt"
    input_lang, output_lang, pairs = readLangs(input_file, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:", input_lang.name, input_lang.n_words, output_lang.name, output_lang.n_words)

    # Split into training and test pairs
    random.shuffle(pairs)
    test_size = int(len(pairs) * test_ratio)
    test_pairs = pairs[:test_size]
    train_pairs = pairs[test_size:]
    return input_lang, output_lang, train_pairs, test_pairs

# Run the updated prepareData function
#input_lang, output_lang, pairs = prepareData(reverse=True)
#print(random.choice(pairs))
input_lang, output_lang, train_pairs, test_pairs = prepareData(reverse=True)
print(random.choice(test_pairs))

Reading lines...
Read 152820 sentence pairs
Trimmed to 9126 sentence pairs
Counting words...
Counted words: eng 4649 deu 3032
['er erblindet', 'he s going blind']


# Architecture

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader_old(batch_size):
    input_lang, output_lang, pairs = prepareData(reverse=True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader


#  [Load Dataloader for Train and Test Sets]
def get_dataloader(pairs, batch_size):
    input_ids = np.zeros((len(pairs), MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((len(pairs), MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    dataset = TensorDataset(torch.LongTensor(input_ids).to(device),
                            torch.LongTensor(target_ids).to(device))
    sampler = RandomSampler(dataset)
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)
    return dataloader



In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,

          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

#  Training & Evaluation

In [ ]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                #decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [ ]:
def run_evaluation(encoder:nn.Module, decoder:nn.Module, pairs:Tuple[str,str], input_lang:Lang, output_lang:Lang, tokenizers=None, print_sentences=False):
    import evaluate as hfeval
    from nltk.translate import gleu_score
    print("Evaluating model on test set:")
    predictions = []
    predictions_tokenized = []
    references = []
    references_tokenized = []
    for pair in pairs:
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        references.append(pair[1])
        references_tokenized.append(pair[1].split(" "))
        output_sentence = ' '.join(output_words)
        predictions.append(output_sentence)
        predictions_tokenized.append(output_words)
        if print_sentences:
          print('>', pair[0])
          print('=', pair[1])
          print(f"<{output_sentence}\n")

    bleu = hfeval.load("bleu")
    #bleu = hfeval.load("bleu", smoothing=True) # same as sacrebleu with default params
    sacrebleu = hfeval.load("sacrebleu") # default smooth_method="exp"

    bleu_score = bleu.compute(predictions=predictions, references=references)
    sacrebleu_score = sacrebleu.compute(predictions=predictions, references=references)
    gleu_number = gleu_score.corpus_gleu(list_of_references = references_tokenized, hypotheses= predictions_tokenized, min_len=1, max_len=4)

    print("Metric scores:")
    print(f"Corpus BLEU Score on test set: {bleu_score}")
    print(f"Corpus SacreBLEU Score on test set: {sacrebleu_score}")
    print(f"Corpus GLEU Score on test set: {gleu_number}")


# Results

In [ ]:
hidden_sizes = [128,192,256]
batch_sizes = [16,32]
num_epochs = [2,8,12]
for hidden_size in hidden_sizes:
    for batch_size in batch_sizes:
        for num_epoch in num_epochs:
            #input_lang, output_lang, train_dataloader = get_dataloader(batch_size)
            train_dataloader = get_dataloader(train_pairs, batch_size)
            test_dataloader = get_dataloader(test_pairs, batch_size)

            encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
            decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

            train(train_dataloader, encoder, decoder, num_epoch, print_every=5, plot_every=5)
            print(f"Model trained for {num_epoch} epochs with hidden size {hidden_size} and batch size {batch_size}")
            encoder.eval()
            decoder.eval()
            run_evaluation(encoder, decoder, test_pairs, input_lang, output_lang)

Model trained for 2 epochs with hidden size 128 and batch size 16
Evaluating model on test set:
Metric scores:
Corpus BLEU Score on test set: {'bleu': 0.07280072530446821, 'precisions': [0.3240149509422208, 0.15793773259508034, 0.04493037423846823, 0.01221664178091489], 'brevity_penalty': 1.0, 'length_ratio': 1.2335030256459514, 'translation_length': 12842, 'reference_length': 10411}
Corpus SacreBLEU Score on test set: {'score': 7.280072530446821, 'counts': [4161, 1740, 413, 90], 'totals': [12842, 11017, 9192, 7367], 'precisions': [32.401495094222085, 15.793773259508033, 4.493037423846824, 1.221664178091489], 'bp': 1.0, 'sys_len': 12842, 'ref_len': 10411}
Corpus GLEU Score on test set: 0.03637901861252115
0m 55s (- 0m 33s) (5 62%) 1.7534
Model trained for 8 epochs with hidden size 128 and batch size 16
Evaluating model on test set:
Metric scores:
Corpus BLEU Score on test set: {'bleu': 0.23346853524420674, 'precisions': [0.4951853429910524, 0.30080726538849645, 0.17971552257266543, 0.1

At first we observed how the metrics Bleu, with ```smooth=True```  and SacreBleu, with ```exp``` smooth method, reported the same number. So decided not to use smoothing in the calculation of BLEU, and the metrics still coincided.

The following tables show the configurations that achieved the top 5 metrics.

| Batch size | Epochs | Hidden size | BLEU / SacreBLEU |
|------------|--------|-------------|------------------|
| 16         | 12     | 256         | 37.5402013095973 |
| 32         | 12     | 256         | 37.0363569555832 |
| 16         | 8      | 256         | 32.9634522496251 |
| 32         | 8      | 256         | 32.7934135542943 |
| 16         | 12     | 192         | 32.7683515125358 |

| Batch size | Epochs | Hidden size | GLEU               |
|------------|--------|-------------|--------------------|
| 32         | 2      | 128         | 0.0394355521869009 |
| 16         | 12     | 256         | 0.0367081483334376 |
| 16         | 2      | 128         | 0.0363790186125212 |
| 32         | 12     | 256         | 0.0353840575330876 |
| 32         | 12     | 192         | 0.0351341029934491 |

For the BLEU metric, we can see the size of the GRU layer is the one that affects the scores the most, followed by the number of epochs.

Using GLEU metric, #1 and #3 configuration somehow obtained while using the lowest number of epochs and smallest hidden size.

